In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [11]:
data = pd.read_csv('Data/properati.csv')
data.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [12]:
round((data.isnull().sum()/data.shape[0]),2)

Unnamed: 0                    0.00
operation                     0.00
property_type                 0.00
place_name                    0.00
place_with_parent_names       0.00
country_name                  0.00
state_name                    0.00
geonames_id                   0.15
lat-lon                       0.43
lat                           0.43
lon                           0.43
price                         0.17
currency                      0.17
price_aprox_local_currency    0.17
price_aprox_usd               0.17
surface_total_in_m2           0.32
surface_covered_in_m2         0.16
price_usd_per_m2              0.43
price_per_m2                  0.28
floor                         0.93
rooms                         0.61
expenses                      0.88
properati_url                 0.00
description                   0.00
title                         0.00
image_thumbnail               0.03
dtype: float64

## Geolocalización.

Hay varios campos que intervienen en la **geolocalización**. Los más útiles son **lan** y **lon** porque nos dan la ubicación exacta de la propiedad. El problema es que tienen un porcentaje alto de nulos, por lo que el objetivo va a ser tratar de inducir estos valores. 

Los campos que vamos a evaluar para definir una estrategia de imputación, serán: 
* **lat-lon:** concatenación de latitud y longitud.
* **lat:** latitud.
* **lon:** longitud.
* **place_name:** localidad o barrio.
* **place_with_parent_names:** concatenación de ubicaciones.
* **country_name:** país.
* **state_name:** provincia o partido.
* **geonames_id:** identificador de ubicación.

### Campos de Coordenadas: lat, lon y lat-lon.
Tenemos tres campos de coordenadas: **lat, lon** y **lat-lon**. Veamos qué información tienen.

In [16]:
data[['lat','lon','lat-lon']]

,lat,lon,lat-lon
0,-34.661824,-58.508839,"-34.6618237,-58.5088387"
1,-34.903883,-57.964330,"-34.9038831,-57.9643295"
2,-34.652262,-58.522982,"-34.6522615,-58.5229825"
3,-34.647797,-58.516424,"-34.6477969,-58.5164244"
4,-38.002626,-57.549447,"-38.0026256,-57.5494468"
...,...,...,...
121215,NaN,NaN,NaN
121216,NaN,NaN,NaN
121217,-34.570639,-58.475596,"-34.5706388726,-58.4755963355"
121218,NaN,NaN,NaN


In [18]:
# Cantidad de nulos
data[['lat','lon','lat-lon']].isnull().sum()

lat        51550
lon        51550
lat-lon    51550
dtype: int64

In [21]:
# Porcentaje de nulos
round(data[['lat','lon','lat-lon']].isnull().sum() / data.shape[0],2)

lat        0.43
lon        0.43
lat-lon    0.43
dtype: float64

A simple vista, pareciera ser que **lat-lon** tiene la misma información que **lat** y **lon**, con un decimal adicional.

Para comprobar si esto efectivamente es así, separaremos **lat-lon** en dos columnas y luego compararemos a cada una contra **lat** y **lon**, respectivamente.

In [13]:
# 1. Separar lat-lon en lat y lon.
lati = data['lat-lon'].apply(lambda x: x if x is np.NaN else float(x.split(",")[0]))
longi = data['lat-lon'].apply(lambda x: x if x is np.NaN else float(x.split(",")[1]))

# 2. Comparar contra lan para ver si son iguales.
lati_notnulls = lati.notnull()
lat_notnulls = data['lat'].notnull()
print('Cantidad no nulos lati:', lati_notnulls.sum())
print('Cantidad no nulos lat:', lat_notnulls.sum())
mask_noiguales_latitud = lati_notnulls != lat_notnulls
data.loc[mask_noiguales_latitud, :]

# 3. Comparar contra lon para ver si son iguales.
longi_notnulls = longi.notnull()
lon_notnulls = data['lon'].notnull()
print('Cantidad no nulos longi:', longi_notnulls.sum())
print('Cantidad no nulos lon:', lon_notnulls.sum())
mask_noiguales_longitud = longi_notnulls != lon_notnulls
data.loc[mask_noiguales_longitud, :]

# 4. Confirmación visual.
data['lat-lon_longitud'] = longi
data['lat-lon_latitud'] = lati

data[['lat-lon', 'lat', 'lat-lon_latitud','lat-lon_longitud', 'lon']]

Cantidad no nulos lati: 69670
Cantidad no nulos lat: 69670
Cantidad no nulos longi: 69670
Cantidad no nulos lon: 69670


,lat-lon,lat,lat-lon_latitud,lat-lon_longitud,lon
0,"-34.6618237,-58.5088387",-34.661824,-34.661824,-58.508839,-58.508839
1,"-34.9038831,-57.9643295",-34.903883,-34.903883,-57.964329,-57.964330
2,"-34.6522615,-58.5229825",-34.652262,-34.652262,-58.522982,-58.522982
3,"-34.6477969,-58.5164244",-34.647797,-34.647797,-58.516424,-58.516424
4,"-38.0026256,-57.5494468",-38.002626,-38.002626,-57.549447,-57.549447
...,...,...,...,...,...
121215,NaN,NaN,NaN,NaN,NaN
121216,NaN,NaN,NaN,NaN,NaN
121217,"-34.5706388726,-58.4755963355",-34.570639,-34.570639,-58.475596,-58.475596
121218,NaN,NaN,NaN,NaN,NaN


Podemos concluir que el campo **lat-lon** tiene la misma información que **lat** y **lon** separadamente. 

Por una cuestión de practicidad es preferible tener estos campos por separado, y por lo tanto se puede prescindir de esta columna.

### Columnas descriptivas de ubicación: place_name, place_with_parent_names, country_name y state_name.

In [22]:
data[['place_name', 'place_with_parent_names', 'country_name', 'state_name']]

,place_name,place_with_parent_names,country_name,state_name
0,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal
1,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur
2,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal
3,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal
4,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica
...,...,...,...,...
121215,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal
121216,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte
121217,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal
121218,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica


In [23]:
data[['place_name', 'place_with_parent_names', 'country_name', 'state_name']].isnull().sum()

place_name                 23
place_with_parent_names     0
country_name                0
state_name                  0
dtype: int64

En estas columnas aparentemente no hay mayores problemas de nulos. Sólo **place_name** tiene 23 elementos a los que les falta valor. 

Revisemos un poco más estos datos.

In [44]:
mask_place_name_nulos = data.place_name.isnull()
mask_place_name_no_nulos = data.place_name.notnull()
print('Registros no nulos:', data.loc[mask_place_name_no_nulos,'place_name'].shape[0])
print('Registros nulos:', data.loc[mask_place_name_nulos,'place_name'].shape[0])
data.loc[mask_place_name_nulos,'place_name']

Registros no nulos: 121197
Registros nulos: 23


6489     NaN
10201    NaN
11451    NaN
14839    NaN
18622    NaN
21922    NaN
23664    NaN
24722    NaN
38856    NaN
45970    NaN
46642    NaN
53130    NaN
55306    NaN
57703    NaN
57759    NaN
57764    NaN
57793    NaN
58004    NaN
58037    NaN
59069    NaN
62411    NaN
62413    NaN
63849    NaN
Name: place_name, dtype: object

In [36]:
data.loc[mask_place_name_nulos, ['place_name', 'place_with_parent_names', 'country_name', 'state_name']]

,place_name,place_with_parent_names,country_name,state_name
6489,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
10201,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
11451,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
14839,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
18622,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
21922,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
23664,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
24722,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
38856,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
45970,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte


Al evaluar los datos se ve que ese dato está faltando y no es posible deducirlo. Como son sólo 23 registros, vamos a eliminarlos de la serie.

In [65]:
place_name_no_nulos = data.loc[mask_place_name_no_nulos,'place_name']
data['place_name_no_nulos'] = place_name_no_nulos
print('Registros sin nulos:', data['place_name_no_nulos'].notnull().sum())
print('Registros nulos:', data['place_name_no_nulos'].isnull().sum())
# REVISAR. No tendría que dar nulos.

Registros sin nulos: 121197
Registros nulos: 23


Al revisar los datos que contiene **place_with_parent_names**, podemos ver que tiene datos que las otras tres columnas juntas no poseen.

In [68]:
data.place_with_parent_names.head(50)

0                 |Argentina|Capital Federal|Mataderos|
1           |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|
2                 |Argentina|Capital Federal|Mataderos|
3                   |Argentina|Capital Federal|Liniers|
4     |Argentina|Buenos Aires Costa Atlántica|Mar de...
5                   |Argentina|Entre Ríos|Gualeguaychú|
6     |Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...
7                  |Argentina|Capital Federal|Belgrano|
8                  |Argentina|Capital Federal|Belgrano|
9                          |Argentina|Santa Fe|Rosario|
10                          |Argentina|Córdoba|Córdoba|
11      |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel|
12    |Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
13     |Argentina|Capital Federal|Palermo|Palermo Soho|
14     |Argentina|Capital Federal|Palermo|Palermo Soho|
15    |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel...
16                |Argentina|Capital Federal|Mataderos|
17    |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|

Vamos a guardar estos datos en columnas separadas para futuros análisis.

### Correspondencia entre las coordenadas y su ubicación.
A continuación vamos a verificar si los puntos están correctamente ubicados. Es decir, si se corresponden con los barrios/localidades que dice el dataset que tiene. 

Hacemos esto para verificar si la información es fiable y se la puede utilizar para complementar otros análisis.

In [ ]:
df = data
df['text'] = df['place_name']

fig = go.Figure(data=go.Scattergeo(
        lon = df['lon'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        ))

fig.update_layout(
        title = 'Propiedades',
        geo_scope="south america",
    )
#fig.show()